In [1]:
import os

In [2]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [5]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class DataIngestionConfig:
    # Configuration class for data ingestion process
    root_dir: Path  # The root directory where data ingestion operations will take place
    source_URL: str  # The URL from which the data will be downloaded
    local_data_file: Path  # The local path where the downloaded data will be stored
    unzip_dir: Path  # The directory where the downloaded data will be unzipped


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get the DataIngestionConfig object from the configuration.
        
        Returns:
            DataIngestionConfig: Configuration for data ingestion.
        """
        config = self.config.data_ingestion  # Get the data ingestion configuration section

        create_directories([config.root_dir])  # Ensure the root directory for data ingestion exists

        # Create and return a DataIngestionConfig object with the configuration details
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize the DataIngestion class with a configuration.
        
        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config  # Store the configuration

    def download_file(self):
        """
        Downloads the file from the source URL if it does not already exist locally.
        """
        if not os.path.exists(self.config.local_data_file):
            # If the file does not exist, download it from the source URL
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            # If the file already exists, log its size
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the unzip directory.
        """
        unzip_path = self.config.unzip_dir  # Get the directory where the zip file will be extracted
        os.makedirs(unzip_path, exist_ok=True)  # Ensure the unzip directory exists
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  # Extract all contents of the zip file into the unzip directory


In [12]:
try:
    config = ConfigurationManager()  # Initialize ConfigurationManager with default file paths
    data_ingestion_config = config.get_data_ingestion_config()  # Retrieve data ingestion configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)  # Initialize DataIngestion with the config
    data_ingestion.download_file()  # Download the data file
    data_ingestion.extract_zip_file()  # Extract the downloaded zip file
except Exception as e:
    raise e  # Raise the exception if any error occurs


[2024-06-11 17:00:17,161: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-11 17:00:17,162: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-11 17:00:17,163: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-11 17:00:17,163: INFO: common: Created directory at: artifacts]
[2024-06-11 17:00:17,164: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-06-11 17:00:17,739: INFO: 1192957093: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1675:3CF6DF:8F95E6:96D06D:66686681
Accept-Ranges: bytes
Date: Tue, 11